In [ ]:
!pip install git+https://github.com/huggingface/transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hzf0b_r9
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-hzf0b_r9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.0 MB 20.6 MB/s 
     |████████████████████████████████| 163 kB 69.2 MB/s 
  Created wheel for transformers: filename=transformers-4.23.0.dev0-py3-none-any.whl size=5100941 sha256=36f61b3d4169d61bfa057ec17034e210b76c22c2fcbe27c8662f8be9862e8df5
  Stored in directory: /tmp/pip-ephem-wheel-cache-hhkzrcjz/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers


In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.3 MB/s 
     |████████████████████████████████| 1.3 MB 58.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9055ad8aeec673bf3f8fca723762a7efc09c909ee455d5422c08b35cd854646f
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
import torch
torch.cuda.empty_cache()

from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
# from transformers import AutoConfig, AutoTokenizer, AutoModel
import logging
from datetime import datetime
import sys
import os
import gzip
import csv
import pandas as pd
import xlrd
import numpy as np
import pandas as pd
from google.colab import files
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sentence_transformers import SentenceTransformer, util


#Load data

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()
!pip install -q xlrd

Saving FaEnDataSetV7_complete.xlsx to FaEnDataSetV7_complete.xlsx


In [ ]:
import pandas as pd
xls = pd.ExcelFile('FaEnDataSetV7_complete.xlsx')

In [ ]:
#convert the dataset to a Dataloader ready for training
train_samples= []
dev_samples= []
test_samples = []
samplePercent=1

df1=pd.read_excel(xls, 'train')
df1=df1.sample(frac=1).reset_index(drop=True)
df2=pd.read_excel(xls, 'dev')
df2=df2.sample(frac=1).reset_index(drop=True)
df3=pd.read_excel(xls, 'test')
df3=df3.sample(frac=1).reset_index(drop=True)


for index, row in df1.iterrows():
  score = float(row['score']) / 5.0
  inp_example=InputExample(texts=[row['farsiSentence'], row['englishSentence']], label=score)
  train_samples.append(inp_example)

for index, row in df2.iterrows():
  score = float(row['score']) / 5.0
  inp_example=InputExample(texts=[row['farsiSentence'], row['englishSentence']], label=score)
  dev_samples.append(inp_example)

for index, row in df3.iterrows():
  score = float(row['score']) / 5.0
  inp_example=InputExample(texts=[row['farsiSentence'], row['englishSentence']], label=score)
  test_samples.append(inp_example)
print('end of reading data')

end of reading data


In [ ]:
scores_train= df1['score'].values.tolist()
gscores_train = [i / 5 for i in scores_train]
english_train = df1['englishSentence'].tolist()
farsi_train= df1['farsiSentence'].tolist()

scores_dev= df2['score'].values.tolist()
gscores_dev = [i / 5 for i in scores_dev]
english_dev = df2['englishSentence'].tolist()
farsi_dev= df2['farsiSentence'].tolist()

scores_test= df3['score'].values.tolist()
gscores_test = [i / 5 for i in scores_test]
english_test = df3['englishSentence'].tolist()
farsi_test= df3['farsiSentence'].tolist()

In [ ]:
scores_train= np.array(scores_train)
scores_dev= np.array(scores_dev)

#Load model

**from drive**

In [ ]:
model_save_path1 = r'/content/drive/MyDrive/Models_Finetuned_Fa-En_3 part dataset/paraphrase-xlm-r-multilingual-v1_dataset(1)'
model1 = SentenceTransformer(model_save_path1)

In [ ]:
model_save_path2 = r'/content/drive/MyDrive/Models_Finetuned_Fa-En_3 part dataset/paraphrase-multilingual-mpnet-base-v2_dataset(2)'
model2 = SentenceTransformer(model_save_path2)

In [ ]:
model_save_path3 = r'/content/drive/MyDrive/Models_Finetuned_Fa-En_3 part dataset/mstsb-paraphrase-multilingual-mpnet-base-v2(3)'
model3 = SentenceTransformer(model_save_path3)

**from hugging face**

In [ ]:
#You can specify any huggingface/transformers pre-trained model here
model_name1='sentence-transformers/paraphrase-xlm-r-multilingual-v1'
model_name2='sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model_name3='AIDA-UPM/mstsb-paraphrase-multilingual-mpnet-base-v2'


word_embedding_model1=models.Transformer(model_name1, max_seq_length=256)
word_embedding_model2=models.Transformer(model_name2, max_seq_length=256)
word_embedding_model3=models.Transformer(model_name3, max_seq_length=256)

pooling_model1=models.Pooling(word_embedding_model1.get_word_embedding_dimension(),
                             pooling_mode_mean_tokens=True)
                             #pooling_mode_cls_tokens=True)
                             #pooling_mode_max_tokens=False)
pooling_model2=models.Pooling(word_embedding_model2.get_word_embedding_dimension(),
                             pooling_mode_mean_tokens=True)
                             #pooling_mode_cls_tokens=True)
                             #pooling_mode_max_tokens=False)
pooling_model3=models.Pooling(word_embedding_model3.get_word_embedding_dimension(),
                             pooling_mode_mean_tokens=True)
                             #pooling_mode_cls_tokens=True)
                             #pooling_mode_max_tokens=False)  '''                                                        
model1=SentenceTransformer(modules=[word_embedding_model1, pooling_model1])
model2=SentenceTransformer(modules=[word_embedding_model2, pooling_model2])
model3=SentenceTransformer(modules=[word_embedding_model3, pooling_model3])

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

#get embedding from models

In [ ]:
#train set

from sentence_transformers import SentenceTransformer, util


#Compute embedding for both lists
embeddings1_train_en = model1.encode(english_train, convert_to_tensor=True)
embeddings1_train_fa = model1.encode(farsi_train, convert_to_tensor=True)

'''#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
scoress=np.array(cosine_scores.cpu()).diagonal()'''


'#Compute cosine-similarits\ncosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)\nscoress=np.array(cosine_scores.cpu()).diagonal()'

In [ ]:
#train set

from sentence_transformers import SentenceTransformer, util


#Compute embedding for both lists
embeddings2_train_en = model2.encode(english_train, convert_to_tensor=True)
embeddings2_train_fa = model2.encode(farsi_train, convert_to_tensor=True)

'''#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
scoress=np.array(cosine_scores.cpu()).diagonal()'''


'#Compute cosine-similarits\ncosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)\nscoress=np.array(cosine_scores.cpu()).diagonal()'

In [ ]:
#train set

from sentence_transformers import SentenceTransformer, util


#Compute embedding for both lists
embeddings3_train_en = model3.encode(english_train, convert_to_tensor=True)
embeddings3_train_fa = model3.encode(farsi_train, convert_to_tensor=True)

'''#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
scoress=np.array(cosine_scores.cpu()).diagonal()'''


'#Compute cosine-similarits\ncosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)\nscoress=np.array(cosine_scores.cpu()).diagonal()'

In [ ]:
#dev set

embeddings1_dev_en = model1.encode(english_dev, convert_to_tensor=True)
embeddings1_dev_fa = model1.encode(farsi_dev, convert_to_tensor=True)

In [ ]:
#dev set

embeddings2_dev_en = model2.encode(english_dev, convert_to_tensor=True)
embeddings2_dev_fa = model2.encode(farsi_dev, convert_to_tensor=True)

In [ ]:
#dev set

embeddings3_dev_en = model3.encode(english_dev, convert_to_tensor=True)
embeddings3_dev_fa = model3.encode(farsi_dev, convert_to_tensor=True)

In [ ]:
#test set

embeddings1_test_en = model1.encode(english_test, convert_to_tensor=True)
embeddings1_test_fa = model1.encode(farsi_test, convert_to_tensor=True)

cosine_scores1 = util.pytorch_cos_sim(embeddings1_test_en, embeddings1_test_fa)
scoress1=np.array(cosine_scores1.cpu()).diagonal()

In [ ]:
#test set

embeddings2_test_en = model2.encode(english_test, convert_to_tensor=True)
embeddings2_test_fa = model2.encode(farsi_test, convert_to_tensor=True)

cosine_scores2 = util.pytorch_cos_sim(embeddings2_test_en, embeddings2_test_fa)
scoress2=np.array(cosine_scores2.cpu()).diagonal()

In [ ]:
#test set

embeddings3_test_en = model3.encode(english_test, convert_to_tensor=True)
embeddings3_test_fa = model3.encode(farsi_test, convert_to_tensor=True)

cosine_scores3 = util.pytorch_cos_sim(embeddings3_test_en, embeddings3_test_fa)
scoress3=np.array(cosine_scores3.cpu()).diagonal()

In [ ]:
ensemble =[(g + h + t) /3  for g, h, t in zip(scoress1, scoress2, scoress3)]

In [ ]:
import scipy.stats
correlation1, p_value = scipy.stats.pearsonr(scoress1, gscores_test)
print(correlation1)

0.9293917988688706


In [ ]:
import scipy.stats
correlation2, p_value = scipy.stats.pearsonr(scoress2, gscores_test)
print(correlation2)

0.9342908499511215


In [ ]:
import scipy.stats
correlation3, p_value = scipy.stats.pearsonr(scoress3, gscores_test)
print(correlation3)

0.9325438577209694


In [ ]:
import scipy.stats
correlation, p_value = scipy.stats.pearsonr(ensemble, gscores_test)
print(correlation)

0.9454058980432131


#concat-embedding of models

In [ ]:
#train
#first concat embedding of 2 model then pca
from sklearn.decomposition import PCA
import scipy.stats
concat_train_en=np.concatenate((embeddings1_train_en.cpu(),embeddings2_train_en.cpu(),embeddings3_train_en.cpu()), axis=1)
print(concat_train_en)
print(len(concat_train_en))
print(len(concat_train_en[0]))

concat_train_fa=np.concatenate((embeddings1_train_fa.cpu(),embeddings2_train_fa.cpu(),embeddings3_train_fa.cpu()), axis=1)
print(concat_train_fa)
print(len(concat_train_fa))
print(len(concat_train_fa[0]))


[[ 0.10365106  0.28248653 -0.21159804 ... -0.15290886  0.06186721
   0.00400199]
 [-0.21046504  0.13614286 -0.20052753 ...  0.0370268  -0.00940119
  -0.04562639]
 [-0.0334396   0.31871334 -0.06339889 ... -0.00751171  0.26351574
   0.02963322]
 ...
 [-0.04820695  0.0805712   0.13013862 ... -0.03600933  0.09727982
  -0.05347791]
 [ 0.18342151  0.20901756 -0.15084657 ... -0.09927318 -0.08957043
  -0.09418356]
 [-0.0547386   0.06868524  0.10300024 ... -0.13482177 -0.1006262
   0.12480208]]
4298
2304
[[ 0.02904925  0.23130065  0.0798125  ... -0.02058543 -0.15282843
   0.01641201]
 [-0.01801092  0.19317447 -0.06241261 ... -0.02318232 -0.01967227
  -0.10053647]
 [ 0.12174065 -0.02989071  0.03696964 ...  0.07767858  0.22171699
   0.05489488]
 ...
 [ 0.18396187 -0.09815053  0.14110465 ... -0.00612094 -0.21834148
  -0.01692173]
 [ 0.02903129  0.20108072 -0.07816893 ... -0.22521113 -0.07083408
   0.03261709]
 [-0.21062976  0.26144993  0.21163045 ... -0.06291737 -0.00611597
   0.13658491]]
4298
23

In [ ]:
#dev
#first concat embedding of 2 model then pca
from sklearn.decomposition import PCA
import scipy.stats
concat_dev_en=np.concatenate((embeddings1_dev_en.cpu(),embeddings2_dev_en.cpu(),embeddings3_dev_en.cpu()), axis=1)
print(concat_dev_en)
print(len(concat_dev_en))
print(len(concat_dev_en[0]))

concat_dev_fa=np.concatenate((embeddings1_dev_fa.cpu(),embeddings2_dev_fa.cpu(),embeddings3_dev_fa.cpu()), axis=1)
print(concat_dev_fa)
print(len(concat_dev_fa))
print(len(concat_dev_fa[0]))



[[-1.23319045e-01  3.35225761e-01  7.37228245e-03 ...  3.60472202e-02
   1.50337413e-01  3.05386167e-03]
 [-7.00930655e-02 -1.01812646e-01  2.77904533e-02 ...  1.55021874e-02
   1.36040777e-01 -1.62011255e-02]
 [-1.66867241e-01  1.11768730e-01 -8.08209926e-02 ...  5.61304204e-02
  -3.52023810e-01  2.12348282e-01]
 ...
 [ 1.85303941e-01 -1.82158113e-01 -2.71133959e-01 ... -8.41404498e-02
   4.90118340e-02 -7.73428679e-02]
 [-2.91511893e-01 -2.48888899e-02 -1.11776285e-01 ... -1.50027379e-04
   1.58611000e-01  6.38746172e-02]
 [ 1.16467290e-01 -1.95886359e-01  2.07468271e-01 ...  2.67546713e-01
  -5.03375642e-02  1.35630190e-01]]
538
2304
[[-0.06997829  0.348829   -0.11803637 ...  0.04600768  0.12645106
   0.04010466]
 [ 0.12174065 -0.02989071  0.03696964 ...  0.07767858  0.22171699
   0.05489488]
 [-0.22577016 -0.02379244 -0.10785016 ...  0.06677192 -0.37268108
   0.13340552]
 ...
 [-0.09974793 -0.26380178 -0.16394962 ... -0.02581928  0.07737142
  -0.09193735]
 [ 0.02911236 -0.23262528 

In [ ]:
#test
#first concat embedding of 2 model then pca
from sklearn.decomposition import PCA
import scipy.stats
concat_test_en=np.concatenate((embeddings1_test_en.cpu(),embeddings2_test_en.cpu(),embeddings3_test_en.cpu()), axis=1)
print(concat_test_en)
print(len(concat_test_en))
print(len(concat_test_en[0]))

concat_test_fa=np.concatenate((embeddings1_test_fa.cpu(),embeddings2_test_fa.cpu(),embeddings3_test_fa.cpu()), axis=1)
print(concat_test_fa)
print(len(concat_test_fa))
print(len(concat_test_fa[0]))

cosine_scoresconcat = util.pytorch_cos_sim(concat_test_en, concat_test_fa)
scoressconcat=np.array(cosine_scoresconcat).diagonal()
correlation, p_value = scipy.stats.pearsonr(scoressconcat, gscores_test)
print(correlation)

'''cosine_scoresconcatpca3 = util.pytorch_cos_sim(concat1, concat2)
scoressconcatpca3=np.array(cosine_scoresconcatpca3).diagonal()
correlation1, p_value = scipy.stats.pearsonr(scoressconcatpca3, gscores)
print(correlation1)'''



[[-0.21346492  0.19202779 -0.04173045 ...  0.02798251 -0.06833222
  -0.12272214]
 [-0.08188949  0.00305683  0.06324567 ...  0.09419691 -0.06613245
  -0.04547225]
 [-0.16126892  0.17416967  0.17460826 ...  0.05408224 -0.02923625
   0.02914008]
 ...
 [ 0.06431075  0.12307905 -0.1398318  ... -0.05922    -0.2333597
   0.0631006 ]
 [-0.07990701 -0.16441779 -0.1973324  ...  0.03667264 -0.12596324
  -0.08332067]
 [ 0.0299635  -0.02756152  0.10893898 ...  0.05745352 -0.03907826
  -0.02820821]]
538
2304
[[-0.09907843  0.19935462  0.07986173 ... -0.04997219  0.11187229
   0.12594852]
 [ 0.07366306  0.12159429 -0.0199703  ...  0.04665314 -0.08319449
  -0.04851718]
 [ 0.17812306  0.7589934  -0.11798245 ...  0.07971254 -0.15808307
   0.07013163]
 ...
 [-0.00097387  0.04330989  0.2433815  ... -0.0829799   0.08510771
   0.20773944]
 [ 0.1502455  -0.09137379 -0.00621627 ...  0.02904177  0.04598492
  -0.05582969]
 [-0.03182101 -0.12005576  0.0185677  ... -0.09126206 -0.01013346
  -0.03149961]]
538
2304

'cosine_scoresconcatpca3 = util.pytorch_cos_sim(concat1, concat2)\nscoressconcatpca3=np.array(cosine_scoresconcatpca3).diagonal()\ncorrelation1, p_value = scipy.stats.pearsonr(scoressconcatpca3, gscores)\nprint(correlation1)'

#concat-PCA-embedding of models-for neural network

In [ ]:
#train
#first concat embedding of 2 model then pca
from sklearn.decomposition import PCA
import scipy.stats
concat_train_en=np.concatenate((embeddings1_train_en.cpu(),embeddings2_train_en.cpu(),embeddings3_train_en.cpu()), axis=1)
print(concat_train_en)
print(len(concat_train_en))
print(len(concat_train_en[0]))

concat_train_fa=np.concatenate((embeddings1_train_fa.cpu(),embeddings2_train_fa.cpu(),embeddings3_train_fa.cpu()), axis=1)
print(concat_train_fa)
print(len(concat_train_fa))
print(len(concat_train_fa[0]))



concatpca = PCA(n_components=350)

concatpca.fit(concat_train_en)
concatpca.fit(concat_train_fa)

concatpca_train_en=concatpca.transform(concat_train_en)
concatpca_train_fa=concatpca.transform(concat_train_fa)


print(len(concatpca_train_en))
print(len(concatpca_train_en[0]))

print(len(concatpca_train_fa))
print(len(concatpca_train_fa[0]))
concat_train_en=concatpca_train_en
concat_train_fa=concatpca_train_fa

[[ 0.1302646  -0.07100274 -0.11750649 ... -0.00945762 -0.1489902
  -0.06965121]
 [-0.0390804   0.40418136  0.03693345 ... -0.02227017  0.1274906
   0.17588298]
 [-0.11269228  0.38375065  0.02486266 ... -0.13912149  0.01513122
   0.01325026]
 ...
 [-0.02922765 -0.16192348 -0.03892796 ...  0.09178364 -0.14742048
   0.1051714 ]
 [-0.2995443   0.26780048  0.01705956 ... -0.02324812  0.0223827
   0.01670646]
 [-0.0840049   0.7181274   0.11711926 ... -0.05575639 -0.17282207
  -0.01797974]]
4298
2304
[[-0.05549566  0.0009401   0.277925   ...  0.06772409 -0.1765735
  -0.14214526]
 [-0.17911263  0.02517593  0.17086433 ... -0.07748895 -0.05742991
  -0.03275171]
 [-0.12911896  0.05352664  0.10625229 ... -0.0715589  -0.04616449
   0.05720006]
 ...
 [-0.00541551 -0.09009491  0.06069462 ...  0.02570609 -0.18891764
   0.15721637]
 [-0.05223003  0.34639344 -0.06525462 ... -0.00864596  0.06390656
   0.00216923]
 [ 0.02113133  0.65703803  0.12584552 ... -0.08077542 -0.17714092
   0.00388809]]
4298
2304


In [ ]:
#dev
#first concat embedding of 2 model then pca
from sklearn.decomposition import PCA
import scipy.stats
concat_dev_en=np.concatenate((embeddings1_dev_en.cpu(),embeddings2_dev_en.cpu(),embeddings3_dev_en.cpu()), axis=1)
print(concat_dev_en)
print(len(concat_dev_en))
print(len(concat_dev_en[0]))

concat_dev_fa=np.concatenate((embeddings1_dev_fa.cpu(),embeddings2_dev_fa.cpu(),embeddings3_dev_fa.cpu()), axis=1)
print(concat_dev_fa)
print(len(concat_dev_fa))
print(len(concat_dev_fa[0]))


concatpca = PCA(n_components=350)

concatpca.fit(concat_dev_en)
concatpca.fit(concat_dev_fa)

concatpca_dev_en=concatpca.transform(concat_dev_en)
concatpca_dev_fa=concatpca.transform(concat_dev_fa)


print(len(concatpca_dev_en))
print(len(concatpca_dev_en[0]))

print(len(concatpca_dev_fa))
print(len(concatpca_dev_fa[0]))
concat_dev_en=concatpca_dev_en
concat_dev_fa=concatpca_dev_fa

[[-0.2513462   0.12684527  0.12876724 ...  0.11519365 -0.18229793
  -0.0901175 ]
 [ 0.14443716  0.42758983  0.1778152  ...  0.01628906 -0.03300113
   0.2088559 ]
 [-0.08105001  0.08093886  0.08190949 ...  0.00876309 -0.08734949
   0.10408434]
 ...
 [ 0.26805055 -0.17469496  0.19898927 ... -0.04999663  0.12192887
  -0.06665138]
 [ 0.10026826 -0.29750428 -0.05111931 ...  0.02262282 -0.06579223
  -0.02960007]
 [-0.01443     0.12855154  0.03284211 ... -0.07201838 -0.06536411
   0.11512217]]
538
2304
[[-0.12614466  0.0515753  -0.02026889 ... -0.03834669 -0.00735359
   0.02495675]
 [ 0.17136301  0.36191243  0.22973442 ... -0.03830286 -0.14005935
   0.171458  ]
 [ 0.06538168  0.11507793 -0.10946633 ...  0.04364917 -0.14028022
   0.09062303]
 ...
 [ 0.17847936 -0.23577298 -0.18597609 ...  0.07146385  0.00115983
  -0.05884512]
 [-0.00107674 -0.32184628  0.0258925  ...  0.02154342 -0.08843008
   0.02642323]
 [-0.11704376  0.03354678  0.15487356 ... -0.03371108 -0.09136986
   0.05716823]]
538
230

In [ ]:
#test
#first concat embedding of 2 model then pca
from sklearn.decomposition import PCA
import scipy.stats
concat_test_en=np.concatenate((embeddings1_test_en.cpu(),embeddings2_test_en.cpu(),embeddings3_test_en.cpu()), axis=1)
print(concat_test_en)
print(len(concat_test_en))
print(len(concat_test_en[0]))

concat_test_fa=np.concatenate((embeddings1_test_fa.cpu(),embeddings2_test_fa.cpu(),embeddings3_test_fa.cpu()), axis=1)
print(concat_test_fa)
print(len(concat_test_fa))
print(len(concat_test_fa[0]))

concatpca = PCA(n_components=350)

concatpca.fit(concat_test_en)
concatpca.fit(concat_test_fa)

concatpca_test_en=concatpca.transform(concat_test_en)
concatpca_test_fa=concatpca.transform(concat_test_fa)


print(len(concatpca_test_en))
print(len(concatpca_test_en[0]))

print(len(concatpca_test_fa))
print(len(concatpca_test_fa[0]))
concat_test_en=concatpca_test_en
concat_test_fa=concatpca_test_fa

'''cosine_scoresconcatpca3 = util.pytorch_cos_sim(concat1, concat2)
scoressconcatpca3=np.array(cosine_scoresconcatpca3).diagonal()
correlation1, p_value = scipy.stats.pearsonr(scoressconcatpca3, gscores)
print(correlation1)'''



[[ 0.00974494  0.01505087 -0.15453075 ...  0.0086109   0.17172706
  -0.03127586]
 [ 0.08368107  0.6935453  -0.03841025 ...  0.00950611 -0.13750483
  -0.02637234]
 [ 0.06741524  0.31580746  0.04822974 ... -0.07210738  0.0241876
   0.09838882]
 ...
 [ 0.11120779 -0.10472161 -0.15324008 ...  0.0646463   0.02293537
   0.01829022]
 [ 0.16320103  0.4081411  -0.2066369  ...  0.15316476  0.01000987
  -0.09857692]
 [-0.00944324  0.09804074 -0.09289841 ...  0.11246478 -0.10539472
  -0.02452226]]
538
2304
[[ 0.06140082  0.00416584  0.05289205 ... -0.0141771  -0.06609654
   0.00576752]
 [ 0.16757104  0.4632598   0.18022124 ... -0.02031048 -0.16807432
  -0.11411131]
 [ 0.01919525  0.3497774   0.063957   ... -0.10262991 -0.02090589
   0.15011968]
 ...
 [-0.00300626  0.07648995 -0.09083453 ...  0.06895331 -0.05512768
   0.00798827]
 [-0.18143493  0.50366545 -0.15305518 ...  0.15001768  0.04549562
  -0.04456733]
 [ 0.14583594 -0.02285581 -0.2631624  ...  0.05020288 -0.04227108
   0.00081131]]
538
2304

'cosine_scoresconcatpca3 = util.pytorch_cos_sim(concat1, concat2)\nscoressconcatpca3=np.array(cosine_scoresconcatpca3).diagonal()\ncorrelation1, p_value = scipy.stats.pearsonr(scoressconcatpca3, gscores)\nprint(correlation1)'

#concat-pca-embedding of models(just for test)_no neural network

In [ ]:
#first concat embedding of 3 models then pca
from sklearn.decomposition import PCA
import scipy.stats
concat_en=np.concatenate((embeddings1_test_en.cpu(),embeddings2_test_en.cpu(),embeddings3_test_en.cpu()), axis=1)
print(concat_en)
print(len(concat_en))
print(len(concat_en[0]))

concat_fa=np.concatenate((embeddings1_test_fa.cpu(),embeddings2_test_fa.cpu(),embeddings3_test_fa.cpu()), axis=1)
print(concat_fa)
print(len(concat_fa))
print(len(concat_fa[0]))



'''cosine_scoresconcatpca3 = util.pytorch_cos_sim(concat1, concat2)
scoressconcatpca3=np.array(cosine_scoresconcatpca3).diagonal()
correlation1, p_value = scipy.stats.pearsonr(scoressconcatpca3, gscores)
print(correlation1)'''

concatpca = PCA(n_components=350)

concatpca.fit(concat_en)
concatpca.fit(concat_fa)

concatpca_en=concatpca.transform(concat_en)
concatpca_fa=concatpca.transform(concat_fa)

print(len(concatpca_en))
print(len(concatpca_en[0]))

print(len(concatpca_fa))
print(len(concatpca_fa[0]))
cosine_scoresconcatpca = util.pytorch_cos_sim(concatpca_en, concatpca_fa)
scoressconcatpca=np.array(cosine_scoresconcatpca).diagonal()
correlation, p_value = scipy.stats.pearsonr(scoressconcatpca, gscores_test)
print(correlation)

[[-0.21346492  0.19202779 -0.04173045 ...  0.02798251 -0.06833222
  -0.12272214]
 [-0.08188949  0.00305683  0.06324567 ...  0.09419691 -0.06613245
  -0.04547225]
 [-0.16126892  0.17416967  0.17460826 ...  0.05408224 -0.02923625
   0.02914008]
 ...
 [ 0.06431075  0.12307905 -0.1398318  ... -0.05922    -0.2333597
   0.0631006 ]
 [-0.07990701 -0.16441779 -0.1973324  ...  0.03667264 -0.12596324
  -0.08332067]
 [ 0.0299635  -0.02756152  0.10893898 ...  0.05745352 -0.03907826
  -0.02820821]]
538
2304
[[-0.09907843  0.19935462  0.07986173 ... -0.04997219  0.11187229
   0.12594852]
 [ 0.07366306  0.12159429 -0.0199703  ...  0.04665314 -0.08319449
  -0.04851718]
 [ 0.17812306  0.7589934  -0.11798245 ...  0.07971254 -0.15808307
   0.07013163]
 ...
 [-0.00097387  0.04330989  0.2433815  ... -0.0829799   0.08510771
   0.20773944]
 [ 0.1502455  -0.09137379 -0.00621627 ...  0.02904177  0.04598492
  -0.05582969]
 [-0.03182101 -0.12005576  0.0185677  ... -0.09126206 -0.01013346
  -0.03149961]]
538
2304


#pca-concat embedding of models(just for test)_no neural network

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from matplotlib import cm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST



pca = PCA(n_components=500)

pca.fit(embeddings1_test_en.cpu())
pca.fit(embeddings1_test_fa.cpu())

embeddingpca1_en=pca.transform(embeddings1_test_en.cpu())
embeddingpca1_fa=pca.transform(embeddings1_test_fa.cpu())

#print(len(embeddingpca1))
#print(len(embeddingpca1[0]))
#cosine_scorespca = util.pytorch_cos_sim(embeddingpca1, embeddingpca2)
# array([0.16178058, 0.08613876], dtype=float32)

In [ ]:
pca = PCA(n_components=500)

pca.fit(embeddings2_test_en.cpu())
pca.fit(embeddings2_test_fa.cpu())

embeddingpca2_en=pca.transform(embeddings2_test_en.cpu())
embeddingpca2_fa=pca.transform(embeddings2_test_fa.cpu())

In [ ]:
pca = PCA(n_components=500)

pca.fit(embeddings3_test_en.cpu())
pca.fit(embeddings3_test_fa.cpu())

embeddingpca3_en=pca.transform(embeddings3_test_en.cpu())
embeddingpca3_fa=pca.transform(embeddings3_test_fa.cpu())

In [ ]:
import numpy.ma as ma

pcaconcat_en=np.concatenate((embeddingpca1_en, embeddingpca2_en,embeddingpca3_en), axis=1)
'''print(pcaconcat1)
print(len(pcaconcat1))
print(len(pcaconcat1[0]))'''

pcaconcat_fa=np.concatenate((embeddingpca1_fa, embeddingpca2_fa,embeddingpca3_fa), axis=1)
'''print(pcaconcat2)
print(len(pcaconcat2))
print(len(pcaconcat2[0]))'''

cosine_scorespcaconcat = util.pytorch_cos_sim(pcaconcat_en, pcaconcat_fa)

In [ ]:
#correlation in pca 50 dimention vectors in 2 model
import scipy.stats
scoresspca=np.array(cosine_scorespca).diagonal()
scoresspca2=np.array(cosine_scorespca2).diagonal()
correlation, p_value = scipy.stats.pearsonr(scoresspca, gscores)
print(correlation)
correlation2, p_value = scipy.stats.pearsonr(scoresspca2, gscores)
print(correlation2)

In [ ]:
#correlation in pca and concat of 50 dimention vectors of 2 model
scoresspcaconcat=np.array(cosine_scorespcaconcat).diagonal()
len(scoresspcaconcat)
correlation, p_value = scipy.stats.pearsonr(scoresspcaconcat, gscores_test)
print(correlation)

0.9371665426096472


#anova

**anova-concat**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
fvalue_Best = SelectKBest(f_classif, k=350)
embeddinganova1_en = fvalue_Best.fit_transform(embeddings1_test_en.cpu(), gscores_test)
embeddinganova1_fa = fvalue_Best.fit_transform(embeddings1_test_fa.cpu(), gscores_test)

embeddinganova2_en = fvalue_Best.fit_transform(embeddings2_test_en.cpu(), gscores_test)
embeddinganova2_fa = fvalue_Best.fit_transform(embeddings2_test_fa.cpu(), gscores_test)

embeddinganova3_en = fvalue_Best.fit_transform(embeddings3_test_en.cpu(), gscores_test)
embeddinganova3_fa = fvalue_Best.fit_transform(embeddings3_test_fa.cpu(), gscores_test)


In [ ]:
import numpy.ma as ma

anovaconcat_en=np.concatenate((embeddinganova1_en, embeddinganova2_en,embeddinganova3_en), axis=1)
'''print(pcaconcat1)
print(len(pcaconcat1))
print(len(pcaconcat1[0]))'''

anovaconcat_fa=np.concatenate((embeddinganova1_fa, embeddinganova2_fa,embeddinganova3_fa), axis=1)
'''print(pcaconcat2)
print(len(pcaconcat2))
print(len(pcaconcat2[0]))'''

cosine_scoresanovaconcat = util.pytorch_cos_sim(anovaconcat_en, anovaconcat_fa)

In [ ]:
#correlation in pca and concat of 50 dimention vectors of 2 model
scoressanovaconcat=np.array(cosine_scoresanovaconcat).diagonal()
len(scoressanovaconcat)
correlation, p_value = scipy.stats.pearsonr(scoressanovaconcat, gscores_test)
print(correlation)

0.5925407909998204


**concat-anova**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
fvalue_Best = SelectKBest(f_classif, k=350)
embeddinganova1_en = fvalue_Best.fit_transform(embeddings1_test_en.cpu(), gscores_test)
embeddinganova1_fa = fvalue_Best.fit_transform(embeddings1_test_fa.cpu(), gscores_test)

embeddinganova2_en = fvalue_Best.fit_transform(embeddings2_test_en.cpu(), gscores_test)
embeddinganova2_fa = fvalue_Best.fit_transform(embeddings2_test_fa.cpu(), gscores_test)

embeddinganova3_en = fvalue_Best.fit_transform(embeddings3_test_en.cpu(), gscores_test)
embeddinganova3_fa = fvalue_Best.fit_transform(embeddings3_test_fa.cpu(), gscores_test)


In [ ]:
#first concat embedding of 3 models then anova

concat_en=np.concatenate((embeddings1_test_en.cpu(),embeddings2_test_en.cpu(),embeddings3_test_en.cpu()), axis=1)
print(concat_en)
print(len(concat_en))
print(len(concat_en[0]))

concat_fa=np.concatenate((embeddings1_test_fa.cpu(),embeddings2_test_fa.cpu(),embeddings3_test_fa.cpu()), axis=1)
print(concat_fa)
print(len(concat_fa))
print(len(concat_fa[0]))



'''cosine_scoresconcatpca3 = util.pytorch_cos_sim(concat1, concat2)
scoressconcatpca3=np.array(cosine_scoresconcatpca3).diagonal()
correlation1, p_value = scipy.stats.pearsonr(scoressconcatpca3, gscores)
print(correlation1)'''

fvalue_Best = SelectKBest(f_classif, k=350)
concatanova_en = fvalue_Best.fit_transform(concat_en, gscores_test)
concatanova_fa = fvalue_Best.fit_transform(concat_fa, gscores_test)

print(len(concatanova_en))
print(len(concatanova_en[0]))

print(len(concatanova_fa))
print(len(concatanova_fa[0]))

cosine_scoresconcatanova = util.pytorch_cos_sim(concatanova_en, concatanova_fa)
scoressconcatanova=np.array(cosine_scoresconcatanova).diagonal()
correlation, p_value = scipy.stats.pearsonr(scoressconcatanova, gscores_test)
print(correlation)

[[-0.21346492  0.19202779 -0.04173045 ...  0.02798251 -0.06833222
  -0.12272214]
 [-0.08188949  0.00305683  0.06324567 ...  0.09419691 -0.06613245
  -0.04547225]
 [-0.16126892  0.17416967  0.17460826 ...  0.05408224 -0.02923625
   0.02914008]
 ...
 [ 0.06431075  0.12307905 -0.1398318  ... -0.05922    -0.2333597
   0.0631006 ]
 [-0.07990701 -0.16441779 -0.1973324  ...  0.03667264 -0.12596324
  -0.08332067]
 [ 0.0299635  -0.02756152  0.10893898 ...  0.05745352 -0.03907826
  -0.02820821]]
538
2304
[[-0.09907843  0.19935462  0.07986173 ... -0.04997219  0.11187229
   0.12594852]
 [ 0.07366306  0.12159429 -0.0199703  ...  0.04665314 -0.08319449
  -0.04851718]
 [ 0.17812306  0.7589934  -0.11798245 ...  0.07971254 -0.15808307
   0.07013163]
 ...
 [-0.00097387  0.04330989  0.2433815  ... -0.0829799   0.08510771
   0.20773944]
 [ 0.1502455  -0.09137379 -0.00621627 ...  0.02904177  0.04598492
  -0.05582969]
 [-0.03182101 -0.12005576  0.0185677  ... -0.09126206 -0.01013346
  -0.03149961]]
538
2304

#concat english and farsi embedding process

#-train set

In [ ]:
#1-concat multiply and subtract

import tensorflow as tf
product_train=np.multiply(concat_train_en, concat_train_fa)
subtract_train=np.subtract(concat_train_en, concat_train_fa)


from sklearn.decomposition import PCA
import scipy.stats
selfconcat_train=np.concatenate((product_train, subtract_train), axis=1)


In [ ]:
print(selfconcat_train)
print(len(selfconcat_train))
print(len(selfconcat_train[0]))

[[ 1.4206311e+00  8.8645583e-03  2.3675270e+00 ...  3.8897896e-01
  -5.4149542e-02 -8.9181781e-02]
 [ 1.5834141e-01  3.3615255e-01  2.2918274e-01 ...  2.2800466e-01
  -3.4450987e-01  3.0648792e-01]
 [ 4.9643453e-02  2.7012753e+00  6.6464633e-02 ...  4.3260887e-02
   8.8852286e-02 -1.2018018e-01]
 ...
 [ 7.9403639e-01  9.1544032e-01  1.9269909e-01 ... -1.7194112e-01
  -3.5233147e-02 -2.2014696e-03]
 [ 7.5107731e-02  2.1066146e+00  2.6847720e-01 ... -3.7154101e-02
   1.0963359e-01 -8.4015869e-02]
 [ 2.0516953e+00  1.1044090e+00  1.2060047e+00 ... -5.9125882e-02
  -2.3962662e-02 -1.8122163e-01]]
4298
700


In [ ]:
#2-just concat
from sklearn.decomposition import PCA
import scipy.stats
selfconcat_train=np.concatenate((concat_train_en.cpu(), concat_train_fa.cpu()), axis=1)
print(selfconcat_train)
print(len(selfconcat_train))
print(len(selfconcat_train[0]))

'''selfconcat2=np.concatenate((embeddings3.cpu(), embeddings4.cpu()), axis=1)
print(concat2)
print(len(concat2))
print(len(concat2[0]))'''

#-dev set

In [ ]:
#1-concat multiply and subtract

import tensorflow as tf
product_dev=np.multiply(concat_dev_en, concat_dev_fa)
subtract_dev=np.subtract(concat_dev_en, concat_dev_fa)


from sklearn.decomposition import PCA
import scipy.stats
selfconcat_dev = np.concatenate((product_dev, subtract_dev), axis=1)


In [ ]:
#2-just concat
from sklearn.decomposition import PCA
import scipy.stats
selfconcat_dev=np.concatenate((concat_dev_en.cpu(), concat_dev_fa.cpu()), axis=1)
print(selfconcat_dev)
print(len(selfconcat_dev))
print(len(selfconcat_dev[0]))

'''selfconcat2=np.concatenate((embeddings3.cpu(), embeddings4.cpu()), axis=1)
print(concat2)
print(len(concat2))
print(len(concat2[0]))'''

#-test set

In [ ]:
#1-concat multiply and subtract

import tensorflow as tf
product_test=np.multiply(concat_test_en, concat_test_fa)
subtract_test=np.subtract(concat_test_en, concat_test_fa)


from sklearn.decomposition import PCA
import scipy.stats
selfconcat_test = np.concatenate((product_test, subtract_test), axis=1)


In [ ]:
#2-just concat
from sklearn.decomposition import PCA
import scipy.stats
selfconcat_test=np.concatenate((concat_test_en.cpu(), concat_test_fa.cpu()), axis=1)
print(selfconcat_test)
print(len(selfconcat_test))
print(len(selfconcat_test[0]))

'''selfconcat2=np.concatenate((embeddings3.cpu(), embeddings4.cpu()), axis=1)
print(concat2)
print(len(concat2))
print(len(concat2[0]))'''

#Feedforward_Neural_Networks_For_Regression & predict on testset

In [ ]:
# Load libraries
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from keras.layers import Dropout

In [ ]:
# Start neural network
network = models.Sequential()

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=4608, activation='relu', input_shape=(selfconcat_train.shape[1],)))
#network.add(Dropout(0.5))

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=4608, activation='relu'))


network.add(layers.Dense(units=2304, activation='relu'))


network.add(layers.Dense(units=2304, activation='relu'))


network.add(layers.Dense(units=700, activation='relu'))


network.add(layers.Dense(units=700, activation='relu'))


network.add(layers.Dense(units=32, activation='relu'))

# Add fully connected layer with no activation function
network.add(layers.Dense(units=1))

In [ ]:
# Compile neural network
network.compile(loss='mse', # Mean squared error
                optimizer='RMSprop', # Optimization algorithm
                metrics=['mse']) # Mean squared error

In [ ]:
# Train neural network

history = network.fit(selfconcat_train, # Features
                      scores_train, # Target vector
                      #validation_split=0.2,
                      epochs=150, # Number of epochs/150
                      verbose=0, # No output
                      batch_size=50, # Number of observations per batch/100
                      validation_data=(selfconcat_dev, scores_dev) # Data for evaluation
)

KeyboardInterrupt: ignored

In [ ]:
yhat = network.predict(selfconcat_test)
yhat
yhat_list = yhat.tolist()

In [ ]:
import itertools
yhat_list=list(itertools.chain.from_iterable(yhat_list))

In [ ]:
correlation, p_value = scipy.stats.pearsonr(yhat_list, scores_test)
correlation

0.9458855202427597

#layer and parameters

In [ ]:
# Create function
def nn_cl_bo2(neurons, activation, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    def nn_cl_fun():
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        if normalization > 0.5:
            nn.add(BatchNormalization())
        for i in range(layers1):
            nn.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            nn.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score


In [ ]:
params_nn2 ={
    'neurons': (32, 4608),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(30, 150),
    'epochs':(100, 170),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

In [ ]:
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons'] = round(params_nn_['neurons'])
optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
             'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
             'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
             'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
params_nn_['optimizer'] = optimizerD[optimizerL[round(params_nn_['optimizer'])]]
params_nn_

In [ ]:
# Fitting Neural Network
def nn_cl_fun():
nn = Sequential()
nn.add(Dense(params_nn_['neurons'], input_dim=10, activation=params_nn_['activation']))
    if params_nn_['normalization'] > 0.5:
nn.add(BatchNormalization())
    for i in range(params_nn_['layers1']):
nn.add(Dense(params_nn_['neurons'], activation=params_nn_['activation']))
    if params_nn_['dropout'] > 0.5:
nn.add(Dropout(params_nn_['dropout_rate'], seed=123))
    for i in range(params_nn_['layers2']):
nn.add(Dense(params_nn_['neurons'], activation=params_nn_['activation']))
nn.add(Dense(1, activation='sigmoid'))
nn.compile(loss='binary_crossentropy', optimizer=params_nn_['optimizer'], metrics=['accuracy'])
    return nn
es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
nn = KerasClassifier(build_fn=nn_cl_fun, epochs=params_nn_['epochs'], batch_size=params_nn_['batch_size'],
                         verbose=0)
 nn.fit(X_train, y_train, validation_data=(X_val, y_val), verbose=1)